In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor
import matplotlib.pyplot as plt

### Importing data

In [2]:
fat_df = pd.read_csv('data/data_clean/fat_data_clean.csv')
kcal_df = pd.read_csv('data/data_clean/kcal_data_clean.csv')
kg_df = pd.read_csv('data/data_clean/kg_data_clean.csv')
protein_df = pd.read_csv('data/data_clean/protein_data_clean.csv')

## Identifying Multicollinearity in Data

In [3]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
# Using fat_df just as preliminary dataset to identify multicollinearity
fat_df = fat_df.drop(['Country'], axis=1)

X = fat_df.iloc[:, :-6]
X_col = X.copy()
X = MinMaxScaler().fit_transform(X)
y = fat_df['Deaths']


vif_data = pd.DataFrame()
vif_data["feature"] = X_col.columns

In [5]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 5)

# Initial results
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X, i)
                          for i in range(len(X_col.columns))]

In [6]:
#Identified that:
# - Vegetable Oil is highly correlative with Vegetable Products attribute
# - Milk and Animal Fats might correlate and fit with Animal Products
vif_data.sort_values('VIF')

,feature,VIF
18,Sugar & Sweeteners_f,1.894525e+00
3,"Aquatic Products, Other_f",3.423765e+00
0,Alcoholic Beverages_f,3.743534e+00
23,Obesity,8.263887e+00
17,Sugar Crops_f,1.079844e+01
9,Miscellaneous_f,1.321336e+02
11,Offals_f,1.421459e+03
22,Vegetables_f,2.050995e+03
15,Starchy Roots_f,3.085332e+03
13,Pulses_f,3.408131e+03


## Using XGBoost for Feature Importance

### Using Food Supply in Kcal dataset

In [7]:
# Removing Country column due to it being categorical and not providing much information
kcal_df = kcal_df.drop(['Country', 'Obesity'], axis=1)
kcal_df.describe()

,Alcoholic Beverages_kcal,Animal Products_kcal,Animal fats_kcal,"Aquatic Products, Other_kcal",Cereals - Excluding Beer_kcal,Eggs_kcal,"Fish, Seafood_kcal",Fruits - Excluding Wine_kcal,Meat_kcal,Milk - Excluding Butter_kcal,...,Treenuts_kcal,Vegetal Products_kcal,Vegetable Oils_kcal,Vegetables_kcal,Undernourished,Confirmed,Deaths,Recovered,Active,Population
count,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,...,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,1.640000e+02
mean,1.341392,9.259680,1.276999,0.002888,20.319704,0.428540,0.607802,2.030773,3.842554,2.956138,...,0.264360,40.741763,4.898168,1.082252,10.902927,2.021972,0.039370,1.452356,0.528685,4.562076e+07
std,1.068834,4.766646,1.299003,0.031438,6.490782,0.308497,0.549763,1.437449,2.198052,2.024847,...,0.289350,4.766396,2.182019,0.653259,11.527729,2.360454,0.048718,1.927617,1.322360,1.591165e+08
min,0.000000,1.623700,0.000000,0.000000,8.956500,0.018800,0.000000,0.147100,0.298000,0.116900,...,0.000000,27.708900,0.932500,0.095700,2.500000,0.000312,0.000000,0.000000,0.000000,5.400000e+04
25%,0.388900,5.031950,0.349750,0.000000,14.734600,0.139350,0.237500,1.210025,2.064850,1.112050,...,0.044125,36.841775,3.107900,0.602000,2.500000,0.140976,0.002013,0.099107,0.010451,2.926500e+06
50%,1.244600,8.933250,0.877500,0.000000,19.619950,0.399150,0.466750,1.746100,3.668800,2.766450,...,0.176050,41.062700,4.682800,1.001300,6.250000,1.011570,0.011998,0.475402,0.074515,1.031600e+07
75%,2.042250,13.160125,1.776900,0.000000,24.828550,0.634925,0.861950,2.386525,5.161625,4.358275,...,0.394200,44.985600,6.538000,1.349850,13.675000,3.487069,0.069503,2.622874,0.347880,3.279175e+07
max,5.157400,22.291100,7.800700,0.400700,37.526500,1.446100,4.418300,8.854000,10.567400,9.944100,...,1.421000,48.386400,10.383900,3.352400,59.599998,10.408199,0.185428,9.039871,8.019819,1.402385e+09


##### Feature Importance for Confirmed Attribute

In [8]:
# Dividing the data between X and y of interest
X = kcal_df.iloc[:, :-6]
X_col = X.copy()
# X = MinMaxScaler().fit_transform(X)
y = kcal_df['Confirmed'] 

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5],
          'max_depth': [3, 4, 5], 
          'min_child_weight': [1, 2, 5, 10]
         }

# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

##### Feature Importance for Deaths Attribute

In [ ]:
# Dividing the data between X and y of interest
X = kcal_df.iloc[:, :-6]
X_col = X.copy()

# Scaled data values by a scale of 100 so XGBoost can learn effectively (issues with small data values)
# Did this since we are only using model for feature importance (would not do for prediction model)
y = kcal_df['Deaths']*100

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5],
          'max_depth': [3, 4, 5],
          'min_child_weight': [1, 2, 5, 10]
         }

# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

##### Feature Importance for Recovered Attribute

In [ ]:
# Dividing the data between X and y of interest
X = kcal_df.iloc[:, :-6]
X_col = X.copy()
# X = MinMaxScaler().fit_transform(X)
y = kcal_df['Recovered']
y.describe()

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5]
#           'max_depth': [3, 4, 5]
#           'min_child_weight': [1, 2, 5, 10]
         }

# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

### Using Food Supply in Kg dataset

In [ ]:
# Removing Country column due to it being categorical and not providing much information
kg_df = kg_df.drop(['Country', 'Obesity'], axis=1)
kg_df

##### Feature Importance for Confirmed Attribute

In [ ]:
# Dividing the data between X and y of interest
X = kg_df.iloc[:, :-6]
X_col = X.copy()
y = kg_df['Confirmed'] 

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5],
          'max_depth': [3, 4, 5],
          'min_child_weight': [1, 2, 5, 10]
         }

# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

##### Feature Importance for Deaths Attribute

In [ ]:
# Dividing the data between X and y of interest
X = kg_df.iloc[:, :-6]
X_col = X.copy()
y = kg_df['Deaths']*100

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5],
          'max_depth': [3, 4, 5],
          'min_child_weight': [1, 2, 5, 10]
         }

# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

##### Feature Importance for Recovered Attribute

In [ ]:
# Dividing the data between X and y of interest
X = kg_df.iloc[:, :-6]
X_col = X.copy()
y = kg_df['Recovered']

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5]
#           'max_depth': [3, 4, 5]
#           'min_child_weight': [1, 2, 5, 10]
         }


# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

### Using Protein Supply dataset

In [ ]:
# Removing Country column due to it being categorical and not providing much information
protein_df = protein_df.drop(['Country', 'Obesity'], axis=1)
protein_df

##### Feature Importance for Confirmed Attribute

In [ ]:
# Dividing the data between X and y of interest
X = protein_df.iloc[:, :-6]
X_col = X.copy()
y = protein_df['Confirmed'] 

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5],
          'max_depth': [3, 4, 5],
          'min_child_weight': [1, 2, 5, 10]
         }


# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

##### Feature Importance for Deaths Attribute

In [ ]:
# Dividing the data between X and y of interest
X = protein_df.iloc[:, :-6]
X_col = X.copy()
y = protein_df['Deaths']*100

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5],
          'max_depth': [3, 4, 5],
          'min_child_weight': [1, 2, 5, 10]
         }

# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()

##### Feature Importance for Recovered Attribute

In [ ]:
# Dividing the data between X and y of interest
X = protein_df.iloc[:, :-6]
X_col = X.copy()
y = protein_df['Recovered']

In [ ]:
# Training-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Training split using K-Fold for cross_val
kfold = KFold(n_splits=10)

# Parameter grid for XGBoost
params = {'gamma': [0.5, 1, 1.5, 2, 5]
#           'max_depth': [3, 4, 5]
#           'min_child_weight': [1, 2, 5, 10]
         }


# Create XGBoost model instance
xgb = XGBRegressor()

# Create GridSearch instance
xgb_gs = GridSearchCV(xgb, param_grid=params)
xgb_gs.fit(X, y)

# Metrics and Cross Validation of model
mse = cross_val_score(xgb_gs, X, y, scoring='neg_mean_squared_error', cv=kfold)
rmse = cross_val_score(xgb_gs, X, y, scoring='neg_root_mean_squared_error', cv=kfold)
mae = cross_val_score(xgb_gs, X, y, scoring='neg_mean_absolute_error', cv=kfold)
r2 = cross_val_score(xgb_gs, X, y, scoring='r2', cv=kfold)

print(f"MSE: {-mse.mean()} ({mse.std()})")
print(f"RMSE: {-rmse.mean()} ({rmse.std()})")
print(f"MAE: {-mae.mean()} ({mae.std()})")

In [ ]:
# Get Feature Importance from model
importance = xgb_gs.best_estimator_.feature_importances_


print("\nFeature Importances:")
for item in zip(X_col.columns, xgb_gs.best_estimator_.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

# Plot feature importance
feat_importance = zip(X_col.columns, importance)
feat_importance = sorted([x for x in feat_importance], key = lambda x: x[1], reverse=True)

plt.bar(*zip(*feat_importance))
plt.xticks(rotation=90)
plt.show()